# 번호판 인식 학습

OpenCV와 tensorflow를 이용한 번호판 인식 프로그램 입니다.

## 모듈 불러오기

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Input, Model

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os

## MNIST 데이터 불러오기 및 정규화

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

## CNN 모델 생성

In [3]:
# CNN 모델을 생성한다.

# Convoultion 한 후 Output size 는 다음과 같이 계산된다.
# (W - F + 2P) / S + 1
# W : 원래 이미지 크기
# F : 커널의 크기
# P : 패딩의 크기 (패딩은 이미지 바깥에 임의의 값 영역을 추가로 주는 것을 의미. 보통 0으로 주는 Zero-Padding 방식을 사용)
# S : Stride의 크기 (커널의 이동 거리. 기본값은 1)

# MaxPooling 한 후 Output size 는 다음과 같이 계산된다.
# (W - M) / S + 1
# W : 원래 이미지의 크기
# M : Pooling 커널의 크기
# S : Stride의 크기 (Pooling 커널의 이동 거리. Convoultion과 다르게 Pooling의 기본 Stride값은 2 이다.)

i = Input(shape=(28, 28, 1)) # 첫번째 층. 입력레이어. 28*28*1의 이미지
x = layers.Conv2D(32, (3, 3), activation='relu')(i) # 3 * 3 커널 32개를 지니는 Convolution 레이어

                                       # 출력 : (28 - 3 + 0) / 1 + 1 = 26 이고 32개의 커널이 존재하므로 최종적으로 26 * 26 * 32 의 output size를 가진다.

x = layers.MaxPooling2D((2, 2))(x) # 각 26 * 26 에서 2 * 2의 크기를 지니는 커널을 이동시켜가며 가장 큰 값을 뽑아내는 Max Pooling 레이어

                                       # 출력 : (26 - 2) / 2 + 1 = 13 이고 들어온 input이 32개 이므로 최종적으로 13 * 13 * 32 의 output size를 가진다.

x = layers.Conv2D(64, (3, 3), activation='relu')(x) # 64개의 3 * 3 커널을 사용하는 Convolution 레이어

# 출력 : (13 - 3 + 0) / 1 + 1 =  11, 결과 : 11 * 11 * 64

x = layers.MaxPooling2D((2, 2))(x)

# 출력 : (11 - 2) / 2 + 1 = 5, 결과 : 5 * 5 * 64

x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# 출력 : (5 - 3 + 0) / 1 + 1 = 3, 결과 : 3 * 3 * 64

x = layers.Flatten()(x) # 완전연결신경망으로 데이터를 전달하기위해서 이미지를 1차원 벡터로 펼쳐주는 층. 9 * 9 * 64 = 5184의 1차원 벡터를 생성함.

x2 = layers.Dense(64, activation='relu')(x) # 64개의 노드를 지니는 일반 층
y = layers.Dense(10, activation='softmax')(x2) # 10개의 노드를 지니는 일반층(class 분류 결과를 보여주기위해 softmax 활성화 함수 사용)
model = Model(inputs=i,outputs=y)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928 

## 모델 컴파일 및 훈련

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1554 - accuracy: 0.9512
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0461 - accuracy: 0.9858
Epoch 3/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0337 - accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0259 - accuracy: 0.9921
Epoch 5/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0225 - accuracy: 0.9927


전이학습 전에 손글씨 테스트

In [14]:
roi = cv.imread("./handwriting/8.png")

roi_gray = cv.cvtColor(roi,cv.COLOR_BGR2GRAY)
thr = cv.adaptiveThreshold(roi_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2) # 이진화 시킴. 방식은 apdative threshold
thr = cv.resize(thr, dsize=(28, 28), interpolation=cv.INTER_AREA)
thr = thr.reshape((28,28,1))
thr = thr / 255.0

np.argmax(model.predict(np.expand_dims(thr,0)))

1/1 [==============================] - 0s 19ms/step


2

## 실제 번호판 데이터 전처리

In [10]:
image_list = os.listdir('./imgs') # imgs라는 폴더 안의 모든 파일명을 가져옵니다.
#print(image_list)

roi_list = [] # 실제 프로그램 내에서 사용될 리스트.
labels = [] # 실제 프로그램 내에서 사용될 target값.

for img in image_list:
  roi = cv.imread('./imgs/'+img) # imgs 폴더에 있는 이미지 하나씩 불러옴
  roi_gray = cv.cvtColor(roi,cv.COLOR_BGR2GRAY) # 흑백으로 변환
  thr = cv.adaptiveThreshold(roi_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2) # 이진화 시킴. 방식은 apdative threshold
  thr = cv.resize(thr, dsize=(28, 28), interpolation=cv.INTER_AREA) # 28 * 28 으로 이미지 크기 변환
  thr = thr / 255.0 # 0~1 사이의 값으로 정규화
  thr = thr.reshape((28,28,1)) # 딥러닝 모듈에 집어넣기 위해서 2차원 흑백이미지를 3차원으로 변환

  roi_list.append(thr) # 리스트에 추가
  labels.append(int(img[0])) # 파일명의 맨 앞글자(숫자)로 라벨을 지정하고 리스트에 추가

train_images2 = np.array(roi_list) # 학습될 이미지 numpy array로 변환
train_labels2 = np.array(labels) # 학습될 라벨 numpy array로 변환

## 전이 학습을 위한 Weight Freezing

In [11]:
x.trainable = False # Convolution Layer들은 학습시키지 않는다.

## MNIST데이터로 학습된 모델에 번호판 데이터로 전이학습

In [12]:
model.fit(train_images2, train_labels2, epochs=5)

Epoch 1/5
20/20 [==============================] - 1s 11ms/step - loss: 0.3625 - accuracy: 0.8959
Epoch 2/5
20/20 [==============================] - 0s 11ms/step - loss: 0.0572 - accuracy: 0.9953
Epoch 3/5
20/20 [==============================] - 0s 11ms/step - loss: 0.0145 - accuracy: 0.9968
Epoch 4/5
20/20 [==============================] - 0s 11ms/step - loss: 0.0318 - accuracy: 0.9968
Epoch 5/5
20/20 [==============================] - 0s 11ms/step - loss: 0.0071 - accuracy: 0.9984


## 임의의 데이터로 성능테스트

In [26]:
roi = cv.imread("./handwriting/0.png")

roi_gray = cv.cvtColor(roi,cv.COLOR_BGR2GRAY)
thr = cv.adaptiveThreshold(roi_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2) # 이진화 시킴. 방식은 apdative threshold
thr = cv.resize(thr, dsize=(28, 28), interpolation=cv.INTER_AREA)
thr = thr.reshape((28,28,1))
thr = thr / 255.0

np.argmax(model.predict(np.expand_dims(thr,0)))

1/1 [==============================] - 0s 24ms/step


0

## 학습한 모델 저장

In [ ]:
model.save('my_cnn_model_new.h5')